A ternary search tree has nodes with the following attributes:
* a character, can be `None`;
* a Boolean flag that indicates whether the character represented
  by this node has been the last in a string that was inserted in the
  tree;
* the "less-than" child;
* the "equals" child and
* the "larger-than" child.

The data structure should support the following operations:
* string insert
* string search
* prefix string search
* return the number of strings stored in the data structure
* return all strings stored in the data structure

Also ensure that an instance of the data structure can be visualy represented, e.g., in aSCII format.

# Implementation

In [114]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


The data structure has been implemented as a class.

In [1]:
from __future__ import (
    annotations,
)
from typing import List

class Node():
    def __init__(self, letter: str, last: bool = False):
        self._letter = letter
        self._left, self._right, self._equal = None, None, None
        self._last = last

    def __repr__(self):
        return self.letter
    
    @property
    def last(self):
        return self._last

    @last.setter
    def last(self, last: str):
        print('setting ', last, ' as last property of the node.')
        self._last = last
    
    @property
    def letter(self):
        return self._letter

    @letter.setter
    def letter(self, letter: str):
        print('setting ', letter, ' as letter of the node.')
        self._letter = letter


    @property
    def left(self):
        return self._left

    @left.setter
    def left(self, left: Node):
        print('setting left neighbor of node ', self,  'to ', left)
        self._left = left

    @property
    def right(self):
        return self._right

    @right.setter
    def right(self, right):
        print('setting right neighbor of node ', self,  'to ', right)
        self._right = right

    @property
    def equal(self):
        return self._equal

    @equal.setter
    def equal(self, equal):
        print('setting equal neighbor of node ', self,  'to ', equal)
        self._equal = equal

    




    

In [54]:
class TernarySearchTree():
    "Ternary search tree is a tree "
    def __init__(self, root: Node = None)-> None:
        self._root = root
        #self._nodes = nodes
        
    @property
    def root(self):
        return self._root

    @root.setter
    def root(self, root):
        print('setting root of tree ', self,  'to ', root)
        self._root = root
      
                
                
    def __len__(self) -> int:  
        """counting the number of words in the tree by countign how many nodes in the tree a last nodes"""
        def get_successessors_rec(self, node: Node, successors:List[Node] = []):
            """recursive function getting all successors of a node and returning them in a list"""
            if node:
                if not node.left and not node.right and not node.equal:
                    return []
                else:
                    if node.right:
                        successors.append(node.right)
                        successors.extend(get_successessors_rec(self, node.right, []))
                    if node.left:
                        successors.append(node.left)
                        successors.extend(get_successessors_rec(self, node.left, []))
                    if node.equal:
                        successors.append(node.equal)
                        successors.extend(get_successessors_rec(self, node.equal, []))
                return successors
            else:
                return []
        
        counter = 0
        successors = get_successessors_rec(self, self.root, [])
        successors.append(self.root)
        #print(successors)
        for node in successors:
            if node:
                if node.last == True:
                    counter += 1
        return counter
            
 
    
    def insert(self, string):
        """inserts the string into the tree"""
        if not self.root:#for an empty tree: create the root
            if string: 
                self.root = Node(string[0])
                current = self.root
                for i, letter in enumerate(string[1:]):
                    to_insert = Node(letter)
                    if i == len(string)-2:
                        print("last")
                        to_insert.last = True
                    current.equal = to_insert
                    current = current.equal
            else:
                self.root = Node("", last = True)
        else:#otherwise insert string into exisitng tree
            current = self.root
            counter = 0
            last = False
            if string: 
                while counter < len(string):
                    if counter == len(string)-1:
                        last = True#inidcating whether we are at the last letter of the word
                    if string[counter] == current.letter:#if we find the letter in the tree
                        if last:#if it is the last letter: make this node a last node
                            current.last = True
                            counter += 1
                        elif current.equal:#otherwise: if node has equal child, move on to equal child
                            current = current.equal
                            counter += 1
                        else:#otherwise insert the next letter from the string as the new equal child
                            counter += 1
                            if counter == len(string)-1:
                                last = True
                            current.equal = Node(string[counter], last)
                            #counter += 1
                            current = current.equal
                    elif string[counter] < current.letter:#if letter of node is smaller than current letter from string: inspect left child
                        if current.left:
                            current = current.left
                        else:
                            current.left = Node(string[counter], last)#if no left child: insert letter as left child
                            current = current.left
                            #counter += 1
                    else:#if letter of node is bigger than current letter from string: inspect right child
                        if current.right:
                            current = current.right
                        else:
                            current.right = Node(string[counter], last)#if no right child: insert letter as right child
                            current = current.right
                            #counter += 1
            else:
                while current.left:
                    current = current.left
                current.left = Node("", last = True)

            
    def search(self, string:str, exact: bool = False) -> bool:
        """searching a string in the ternary tree, returns False if string not found and True if found"""
        if not self.root:#if tree empty return False
            return False
        current = self.root
        counter = 0
        while counter < len(string):
            if string[counter] == current.letter:#if we find the letter in the tree

                if current.equal and counter != len(string)-1:#and it is not the last letter of the string
                    current = current.equal#move on to equal node
                    counter += 1
                elif counter == len(string)-1:#if it last letter increase counter by 1 and pass
                    counter += 1
                    pass
                else:
                    return False#if no equal child and not last letter of string: string is not in teh tree
            elif string[counter] < current.letter:#if letter smaller than letter of node
                if current.left and counter != len(string)-1:
                    current = current.left#move on to left child
                else:
                    return False
            else:
                if current.right:
                    current = current.right
                else:
                    return False
        
        if exact == False:
            return True
        else:
            print("current", current.last)
            if current.last == True:
                return True
            else:
                return False
    
    def all_strings(self) -> List[str]:
        def get_all_strings_rec(current: Node, word: str):
            strings = []
            if current.left:
                strings.extend(get_all_strings_rec(current.left, word))
            if current.right: 
                 strings.extend(get_all_strings_rec(current.right, word))
            if current.equal:
                strings.extend(get_all_strings_rec(current.equal, word+current.letter))
            if current.last:
                strings.extend([word+current.letter])
            return strings

        if not self.root:
            return None
        current = self.root
        strings = get_all_strings_rec(current, "")
        return strings

                
                

In [55]:
t = TernarySearchTree()

In [56]:
len(t)

0

In [57]:
t.all_strings()

In [58]:
t.insert("hello")

setting root of tree  <__main__.TernarySearchTree object at 0x7fbf22718910> to  h
setting equal neighbor of node  h to  e
setting equal neighbor of node  e to  l
setting equal neighbor of node  l to  l
last
setting  True  as last property of the node.
setting equal neighbor of node  l to  o


In [59]:
t.all_strings()

['hello']

In [60]:
len(t)

1

In [61]:
t.insert("hel")
t.insert("abc")

setting  True  as last property of the node.
setting left neighbor of node  h to  a
setting equal neighbor of node  a to  b
setting equal neighbor of node  b to  c
setting  True  as last property of the node.


In [62]:
len(t)

3

In [63]:
t.all_strings()

['abc', 'hello', 'hel']

In [64]:
t.search("helo")



False

In [65]:
t.search("hel")

True

In [66]:
t.search("hello")

True

In [67]:
t.search("hello", exact = True)

current True


True

In [68]:
t.search("hel", exact = True)

current True


True

In [69]:
t.search("hell")

True

In [70]:
t.search("hell", exact = True)

current False


False

In [71]:
t.search("he", exact = True)

current False


False

In [72]:
t.search("he")

True

In [73]:
t.insert("")

setting left neighbor of node  a to  


In [74]:
len(t)

4

In [23]:
string = "a"

In [24]:
if not string:
    print(True)

In [25]:
print(t)

In [75]:
t.all_strings()

['', 'abc', 'hello', 'hel']

# Example usage

Create a new empty ternery search tree.

In [27]:
tst = TernarySearchTree()

Insert the string `'abc'` into the tree.

In [28]:
tst.insert('abc')

Display the tree.

In [29]:
print(tst)

terminates: False
       char: a, terminates: False
_eq:      char: b, terminates: False
_eq:        char: c, terminates: True


Insert another string `'aqt'`.

In [30]:
tst.insert('aqt')

In [31]:
print(tst)

terminates: False
       char: a, terminates: False
_eq:      char: b, terminates: False
_eq:        char: c, terminates: True
_gt:        char: q, terminates: False
_eq:          char: t, terminates: True


The tree should now contain two strings.

In [32]:
len(tst)

2

In [33]:
tst.all_strings()

['abc', 'aqt']

Search for the string `'ab'`, it should be found since it is a prefix of `'abc'`.

In [34]:
tst.search('ab')

True

The string `'ac'` should not be found.

In [35]:
tst.search('ac')

False

The tree can also contain the empty string.

In [36]:
tst.insert('')

In [37]:
len(tst)

3

In [38]:
print(tst)

terminates: True
       char: a, terminates: False
_eq:      char: b, terminates: False
_eq:        char: c, terminates: True
_gt:        char: q, terminates: False
_eq:          char: t, terminates: True


In [39]:
tst.all_strings()

['', 'abc', 'aqt']

# Testing

The file `data/search_trees/insert_words.txt` contains words that we can insert into a tree.

In [64]:
tst = TernarySearchTree()
with open('data/search_trees/insert_words.txt') as file:
    words = [
        line.strip() for line in file
    ]
for word in words:
    tst.insert(word)
unique_words = set(words)

setting root of tree  <__main__.TernarySearchTree object at 0x7fd35c2c6830> to  c
setting equal neighbor of node  c to  o
setting equal neighbor of node  o to  m
setting equal neighbor of node  m to  b
setting equal neighbor of node  b to  i
setting equal neighbor of node  i to  n
last
setting  True  as last property of the node.
setting equal neighbor of node  n to  e
setting left neighbor of node  e to  a
setting right neighbor of node  a to  t
setting left neighbor of node  t to  i
setting right neighbor of node  i to  o
setting left neighbor of node  o to  n
setting right neighbor of node  n to  s
setting equal neighbor of node  a to  t
setting left neighbor of node  t to  i
setting right neighbor of node  i to  o
setting left neighbor of node  o to  n
setting equal neighbor of node  e to  d
setting right neighbor of node  d to  s
setting right neighbor of node  c to  d
setting right neighbor of node  d to  u
setting left neighbor of node  u to  c
setting right neighbor of node  c 

Verify the length of the data stucture.

In [70]:
#assert len(tst) == len(unique_words), \
print(f'{len(tst)} in tree, expected {len(unique_words)}')

20 in tree, expected 20


In [73]:
unique_words

{'a',
 'bomb',
 'combination',
 'combinations',
 'combine',
 'combined',
 'combines',
 'duck',
 'ducked',
 'ducks',
 'far',
 'font',
 'fontain',
 'futile',
 'futility',
 'future',
 'the',
 'their',
 'there',
 'therefor'}

Verify that all words that were inserted can be found.

In [72]:
for word in unique_words:
    assert tst.search(word), f'{word} not found'

letter c
node c
0
letter o
node o
1
letter m
node m
2
letter b
node b
3
letter i
node i
4
letter n
node n
5
letter e
node e
6
letter s
node s
7
letter t
node t
0
letter h
node h
1
letter e
node e
2
letter r
node r
3
letter e
node e
4


AssertionError: therefor not found

Verify that all prefixes can be found.

In [43]:
for word in unique_words:
    for i in range(len(word) - 1, 0, -1):
        prefix = word[:i]
        assert tst.search(prefix), f'{prefix} not found'

Chack that when searching for a exact match, only the inserted words are found, and no prefixes.

In [44]:
for word in unique_words:
    for i in range(len(word), 0, -1):
        prefix = word[:i]
        if prefix not in unique_words:
            assert not tst.search(prefix, exact=True), \
                   f'{prefix} found'

Check that the empty string is in the tree (since it is a prefix of any string).

In [45]:
assert tst.search(''), 'empty string not found'

Check that the empty string is not in the tree for an exact search.

In [46]:
assert not tst.search('', exact=True), 'empty string found'

Check that words in the file `data/search_trees/not_insert_words.txt` can not be found in the tree.

In [47]:
with open('data/search_trees/not_insert_words.txt') as file:
    for line in file:
        word = line.strip()
        assert not tst.search(word), f'{word} should not be found'

Check that all strings are returned.

In [48]:
all_strings = tst.all_strings()
assert len(all_strings) == len(unique_words), \
       f'{len(all_strings)} words, expected {len(unique_words)}'
assert sorted(all_strings) == sorted(unique_words), 'words do not match'

If not output was generated, all tests have passed.